In [10]:
import numpy as np
#import matplotlib.pyplot as plt

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout # GRU, Bidirectional
#from keras.optimizer import SGD
import math
from sklearn.metrics import mean_squared_error

In [11]:
data = pd.read_csv(r"C:\Users\raksh\Downloads\IBM_2006-01-01_to_2018-01-01.csv", index_col='Date', parse_dates=['Date'])
data.head() #view first five columns

,Open,High,Low,Close,Volume,Name
Date,,,,,,
2006-01-03,82.45,82.55,80.81,82.06,11715200,IBM
2006-01-04,82.20,82.50,81.33,81.95,9840600,IBM
2006-01-05,81.40,82.90,81.00,82.50,7213500,IBM
2006-01-06,83.95,85.03,83.41,84.95,8197400,IBM
2006-01-09,84.10,84.25,83.38,83.73,6858200,IBM


In [12]:
mytrain = data[:'2016'].iloc[:,1:2].values #selecting all rows upto 2016 and selecting column 1
mytest = data['2017':].iloc[:,1:2].values #Tries to predict all rows after 2017 and selecting column 1

In [13]:
# Scaling the training set
sc = MinMaxScaler(feature_range=(0,1)) # MinMaxScaler(feature_range= (start,stop))
mytrain_scaled = sc.fit_transform(mytrain) #instance.fit_transform(data)

In [14]:
mytrain_scaled #view the scaled values!
#82.55 ==> 0.06065...

array([[0.06065089],
       [0.06029868],
       [0.06311637],
       ...,
       [0.66074951],
       [0.65546633],
       [0.6534235 ]])

In [15]:
len(mytrain_scaled)

2769

In [20]:
I_train = []
O_train = []
for i in range(60,2769):
    I_train.append(mytrain_scaled[i-60:i,0]) #every sequence will have 60 rows/values as input
    O_train.append(mytrain_scaled[i,0])

In [21]:
I_train[0]

array([0.06065089, 0.06029868, 0.06311637, 0.0781206 , 0.07262609,
       0.07171034, 0.07657087, 0.07058326, 0.0669907 , 0.06494787,
       0.075796  , 0.07361229, 0.06417301, 0.05621302, 0.05783319,
       0.05409975, 0.05431107, 0.05515638, 0.05543815, 0.05677656,
       0.05846717, 0.05388842, 0.04811214, 0.04233587, 0.04402649,
       0.0490279 , 0.04832347, 0.05297267, 0.05614258, 0.05290223,
       0.05325444, 0.04909834, 0.04994365, 0.04797126, 0.05431107,
       0.05212736, 0.04726684, 0.04895745, 0.04656241, 0.04839391,
       0.04416737, 0.0485348 , 0.04719639, 0.04825303, 0.05395886,
       0.05663567, 0.05853762, 0.05959425, 0.06375035, 0.06917442,
       0.06889265, 0.06670893, 0.06910397, 0.07783883, 0.07565511,
       0.07276698, 0.06889265, 0.0656523 , 0.06656805, 0.06769513])

In [22]:
O_train[0]

0.06875176105945335

In [23]:
I_train = np.array(I_train) #converting into arrays!
O_train = np.array(O_train)

In [24]:
I_train.shape #samples,steps

(2709, 60)

In [25]:
I_train= I_train.reshape(2709,60,1)
#no. of sequences, inputs in every sequence, no. of features

In [26]:
I_train.shape

(2709, 60, 1)

In [27]:
#building the model

model = Sequential()
# first LSTM layer
model.add(LSTM(units=50, return_sequences=True, input_shape=(I_train.shape[1],1)))
model.add(Dropout(0.2))
# second LSTM layer
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
# third LSTM layer
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
# fourth LSTM layer
model.add(LSTM(units=50))
model.add(Dropout(0.2))
# the output layer
model.add(Dense(units=1))

C:\Users\raksh\anaconda4\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [29]:
#compile
model.compile(optimizer='rmsprop',loss='mean_squared_error')

In [30]:
model.fit(I_train,O_train,epochs=50,batch_size=32)

Epoch 1/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 20s 98ms/step - loss: 0.0494
Epoch 2/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 8s 98ms/step - loss: 0.0108
Epoch 3/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 8s 97ms/step - loss: 0.0085
Epoch 4/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 9s 101ms/step - loss: 0.0074
Epoch 5/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 8s 95ms/step - loss: 0.0064
Epoch 6/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 8s 96ms/step - loss: 0.0063
Epoch 7/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 8s 94ms/step - loss: 0.0056
Epoch 8/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 8s 93ms/step - loss: 0.0049
Epoch 9/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 8s 95ms/step - loss: 0.0046
Epoch 10/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 8s 97ms/step - loss: 0.0045
Epoch 11/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 8s 96ms/step - loss: 0.0043
Epoch 12/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 8s 98ms/step - loss: 0.0036
Epoch 13/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 8s 98ms/step - loss: 0.0034
Epoch 14/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 8s 97ms/step - loss: 0.0034
Epoch 15/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 8s 98ms/step - loss: 0.0037
Ep

In [32]:
model.save("StockIBM.h5")

In [38]:
#test
from tensorflow.keras.models import load_model
import numpy as np
from sklearn.preprocessing import MinMaxScaler
#load the model
model = load_model('StockIBM.h5')
user_input = [83.58746017, 80.34122742, 80.81285462, 82.00291895, 83.29159961, 81.44688209, 80.76357678, 83.80165525, 82.51120409, 82.48569342, 80.65383659, 81.69740582, 82.78338862, 83.37704145, 83.53829805, 80.20374715, 80.42476455, 82.69989097, 83.73417143, 82.95666579, 83.94381372, 82.37856671, 83.60787851, 81.80850007, 82.61239819, 82.06021048, 81.0673498, 83.7835985, 81.74964384, 83.10783276, 80.54146548, 80.49215929, 82.03237911, 81.79802128, 80.93914228, 83.90444224, 83.2143073, 81.31844381, 83.23807452, 83.4603476, 81.85039395, 81.3031393, 80.6351211, 83.65161771, 80.54893536, 82.22940581, 82.50289209, 80.59368723, 83.24620825, 81.77694404, 83.20275117, 83.43996357, 83.38335394, 82.69282925, 80.80822433, 80.30044039, 83.6842126, 82.82960781, 80.12425038, 82.25607533]

user_input_array = np.array(user_input).reshape(-1, 1)
sc = MinMaxScaler(feature_range=(0,1))
user_scaled = sc.fit_transform(user_input_array)
user_scaled = user_scaled.reshape(1, 60, 1)

pred = model.predict(user_scaled)
print(pred)  # Changed pred(pred) to print(pred)
#up-scale
pred_original = sc.inverse_transform(pred)
print("The stock price is",pred_original[0][0]) #up scaled/original o/p

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[[0.5179317]]
The stock price is 82.10252
